In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

import pandas as pd
import seaborn as sns

import sys
sys.path.append("../../")

from gMLV import *
#import importlib
# import gMLV_ML
# import gMLV_sim
#importlib.reload(gMLV_ML);
#importlib.reload(gMLV_sim);

In [5]:
# some plotting functions
def plot_fit_gLV(yobs, y0, nsp, m_h, M_h):
    # plot the fit
    cols = ["red", "green", "blue", "royalblue","black"]
    #cols = [1,2,3,4,5]
    yobs_pred = odeint(gLV, y0, times, args=(nsp, mu_h, M_h))
    for i in range(nsp):
        plt.plot(times, yobs[:, i], color=cols[i])
        plt.plot(times, yobs_pred[:, i], '--', color=cols[i])

def plot_params(mu_h, M_h, e_h ):
    print("\ninferred params:")
    print("mu_hat/mu:")
    print(np.array(mu_h))
    print("\nM_hat/M:")
    print(np.round(np.array(M_h),decimals=2))
    print("e_hat/e:")
    print(np.array(e_h))

    # plot the params
    plt.figure(figsize=(6.4*3,4.8))
    plt.subplot(1, 3, 1)
    plt.stem(np.arange(0,nsp, dtype="int32"), np.array(mu_h), markerfmt="D")

    plt.subplot(1, 3, 2)
    plt.stem(np.arange(0,nsp*nsp), np.array(M_h).flatten(), markerfmt="D")
    
    plt.subplot(1, 3, 3)
    plt.stem(np.arange(0,nsp), np.array(e_h), markerfmt="D")
    

## Repeat Stein et al. 2013 analysis


In [8]:
nsp = 11
# read in 
F = pd.read_csv('Stein_example_F.csv', delimiter = ',')
X = pd.read_csv('Stein_example_Y.csv', delimiter = ',')

#print(F)
#print(X)

F = F.drop(F.columns[[0, 1]], axis=1) 
X = X.drop(X.columns[[0, 1]], axis=1) 

colnames = F.columns.to_list()

#print(F.shape)
#print(X.shape)

# get the best lambda/alpha values on a grid via cross validation
a0, a1, a2 = fit_alpha_Ridge2(X.to_numpy(), F.to_numpy(), num_species=nsp, n_a0=10, n_a1=10, n_a2=10)

# do final fit
mu_h, M_h, e_h = do_final_fit_Ridge2(X.to_numpy(), F.to_numpy(), nsp, a0=a0, a1=a1, a2=a2)

predictor = gMLV_sim(num_species=num_species,
                     num_metabolites=num_metabolites,
                     M=M_h,
                     mu=mu_h,
                     epsilon=e_h)

# this does the stem plots with orange crosses the actual parameters
plot_params(mu_h, M_h, e_h)

# plot interaction matrix
plt.figure(figsize=(6.4*2,6.4*2))
lims = np.max( [np.max(M_h), np.abs(np.min(M_h)) ] )
sns.heatmap(M_h, xticklabels=colnames, yticklabels=colnames, square=True, vmin=-lims, vmax=lims, annot=True, fmt=".2f", cmap='coolwarm');


TypeError: fit_alpha_Ridge2() missing 1 required positional argument: 'num_pert'